<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [1]:
!wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

--2025-03-01 19:18:46--  https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.43.25, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/1199620/files/SO_vectors_200.bin [following]
--2025-03-01 19:18:47--  https://zenodo.org/records/1199620/files/SO_vectors_200.bin
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 1453905423 (1.4G) [application/octet-stream]
Saving to: ‘SO_vectors_200.bin?download=1’

SO_vectors_200.bin? 100%[===================>]   1.35G  29.1MB/s    in 53s     

2025-03-01 19:19:40 (26.0 MB/s) - ‘SO_vectors_200.bin?download=1’ saved [1453905423/1453905423]



In [2]:
from gensim.models.keyedvectors import KeyedVectors
wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [3]:
word = 'dog'
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [4]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### ***Вопрос 1:***
* Входит ли слово `cat` в топ-5 близких слов к слову `dog`? Какое место оно занимает?


In [5]:
# method most_simmilar
wv_embeddings.most_similar('dog')[:5]

[('animal', 0.8564180135726929),
 ('dogs', 0.7880866527557373),
 ('mammal', 0.7623804211616516),
 ('cats', 0.7621253728866577),
 ('animals', 0.760793924331665)]

#### ***Ваш ответ:***

**Нет**, не входит. <br>
Но входит слово `cats` на 4-ой позиции

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

In [6]:
import numpy as np
import re
from nltk.tokenize import word_tokenize, WordPunctTokenizer

class MyTokenizer:
    def __init__(self):
        pass
    def tokenize(self, text):
        return re.findall('\w+', text)


tokenizer = MyTokenizer()
test_text = "This is a test text. Something more meaningfull should be here, but kind of not."
tokenizer.tokenize(test_text)

['This',
 'is',
 'a',
 'test',
 'text',
 'Something',
 'more',
 'meaningfull',
 'should',
 'be',
 'here',
 'but',
 'kind',
 'of',
 'not']

In [7]:
for word in tokenizer.tokenize(test_text):
    if word in wv_embeddings:
        print(word, '->', wv_embeddings[word].shape)
    else:
        print(word, '->', "not in dictionary")

This -> not in dictionary
is -> not in dictionary
a -> not in dictionary
test -> (200,)
text -> (200,)
Something -> not in dictionary
more -> not in dictionary
meaningfull -> (200,)
should -> not in dictionary
be -> not in dictionary
here -> not in dictionary
but -> not in dictionary
kind -> (200,)
of -> not in dictionary
not -> not in dictionary


In [8]:
def question_to_vec(question, embeddings, tokenizer, dim=200):
    """
        question: строка
        embeddings: наше векторное представление
        dim: размер любого вектора в нашем представлении

        return: векторное представление для вопроса
    """

    # Инициализируем вектор вопроса нулями
    q_vector = np.zeros(shape=(dim,))
    # Распарсим вопрос на отдельные слова
    q_words = tokenizer.tokenize(question)
    # Для слов, которые есть в словаре, считаем векторы из эмбеддингов и суммируем их
    for i in q_words:
        if i in embeddings:
            q_vector += embeddings[i]

    return q_vector

Теперь у нас есть метод для создания векторного представления любого предложения.

#### ***Вопрос 2:***

* Какая третья (с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [10]:
# Предложение
question = "I love neural networks"

my_tokenizer = MyTokenizer()
vector = question_to_vec(question, tokenizer=my_tokenizer, embeddings=wv_embeddings)
print(len(vector))
vector[:5]

200


array([-3.04268247, -5.06737871, -3.85623652, -4.1130904 ,  0.47749902])

#### **Ответ 2:**

**-3.86**

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями.

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

Hits@K  измеряет долю вопросов, для которых правильный ответ попал в топ-K позиций среди отранжированных кандидатов.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции::
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа.

DCG@K  измеряет качество ранжирования, учитывая не только факт наличия правильного ответа в топ-K, но и ***его точную позицию***.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера.
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1]$

Проверяем условие $ \text{rank}_{q'_1} \leq 1 $: ***условие неверно***.

Следовательно, $[\text{rank}_{q'_1} \leq 1] = 0$.

- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Проверяем условие $ \text{rank}_{q'_1} \leq 4 $: ***условие верно***.

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### ***Вопрос 3***:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

In [11]:
import numpy as np

ans = 1 / np.log2(10)
print(f"DSG@10(rank_q=9) = {round(ans, 1)}")

DSG@10(rank_q=9) = 0.3



#### **Ответ 3:**

 **DCG@10(rank_q=9) = 0.3**

#### Более сложный пример оценок

Рассмотрим пример с $ N > 1 $, где $ N = 3 $ (три вопроса) и для каждого вопроса заданы позиции их дубликатов. Вычислим метрики **Hits@K** для разных значений $ K $.

---

- $ N = 3 $: Три вопроса ($ q_1, q_2, q_3 $).
- Для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):
  - $ \text{rank}_{q'_1} = 2 $,
  - $ \text{rank}_{q'_2} = 5 $,
  - $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **Hits@K** для $ K = 1, 5 $.

---

**Для $ K = 1 $:**

Подставим значения:
$$
\text{Hits@1} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 1] + [\text{rank}_{q'_2} \leq 1] + [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{Hits@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$

$$
\boxed{\text{Hits@1} = \frac{1}{3}}.
$$

---

**Для $ K = 5 $:**

Подставим значения:
$$
\text{Hits@5} = \frac{1}{3} \cdot \left( [\text{rank}_{q'_1} \leq 5] + [\text{rank}_{q'_2} \leq 5] + [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{Hits@5} = \frac{1}{3} \cdot (1 + 1 + 1) = 1.
$$

$$
\boxed{\text{Hits@5} = 1}.
$$

Теперь вычислим метрику **DCG@K** для того же примера, где $ N = 3 $ (три вопроса), и для каждого вопроса известна позиция его дубликата ($ \text{rank}_{q'_i} $):

- $ \text{rank}_{q'_1} = 2 $,
- $ \text{rank}_{q'_2} = 5 $,
- $ \text{rank}_{q'_3} = 1 $.

Мы будем вычислять **DCG@K** для $ K = 1, 5 $.

---
**Для $ K = 1 $:**
Подставим значения:
$$
\text{DCG@1} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 1] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 1] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 1 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \not\leq 1 $ → $ 0 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 1 $ → $ 1 $.

Сумма:
$$
\text{DCG@1} = \frac{1}{3} \cdot (0 + 0 + 1) = \frac{1}{3}.
$$
$$
\boxed{\text{DCG@1} = \frac{1}{3}}.
$$

---


**Для $ K = 5 $:**
Подставим значения:
$$
\text{DCG@5} = \frac{1}{3} \cdot \left( \frac{1}{\log_2(1 + \text{rank}_{q'_1})} \cdot [\text{rank}_{q'_1} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_2})} \cdot [\text{rank}_{q'_2} \leq 5] + \frac{1}{\log_2(1 + \text{rank}_{q'_3})} \cdot [\text{rank}_{q'_3} \leq 5] \right).
$$

Проверяем условие $ \text{rank}_{q'_i} \leq 5 $ для каждого вопроса:
- $ \text{rank}_{q'_1} = 2 $ → $ 2 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_2} = 5 $ → $ 5 \leq 5 $ → $ 1 $,
- $ \text{rank}_{q'_3} = 1 $ → $ 1 \leq 5 $ → $ 1 $.

Сумма:
$$
\text{DCG@5} = \frac{1}{3} \cdot (0.631 + 0.387 + 1) = \frac{1}{3} \cdot 2.018 \approx 0.673.
$$

$$
\boxed{\text{DCG@5} \approx 0.673}.
$$

#### ***Вопрос 4:***
* Найдите максимум `Hits@47 - DCG@1`?

#### Ответ 4
max(Hits@47 - DCG@1) = 1

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$.

$dup\_ranks$ является списком, который содержит рейтинги дубликатов (их позиции в ранжированном списке).

К примеру для <font color='red'>"Что такое язык python?"</font> $dup\_ranks = [2]$.

In [12]:
def hits_count(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть  Hits@k
    """
    hits_value = 0
    for i in dup_ranks:
        if i <= k:
            hits_value += 1

    hits_value = hits_value / len(dup_ranks)
    return hits_value

In [13]:
dup_ranks = [2]

k = 1
hits_value = hits_count(dup_ranks, k)
print(f"Hits@1 = {hits_value}")

k = 4
hits_value = hits_count(dup_ranks, k)
print(f"Hits@4 = {hits_value}")

Hits@1 = 0.0
Hits@4 = 1.0


In [14]:
import math

def dcg_score(dup_ranks, k):
    """
        dup_ranks: list индексов дубликатов
        result: вернуть DCG@k
    """
    dcg_value = 0
    for i in dup_ranks:
        if i <= k:
            dcg_value += 1 / np.log2(1 + i)

    dcg_value = dcg_value / len(dup_ranks)
    return dcg_value

In [15]:
# Пример списка позиций дубликатов
dup_ranks = [2]

# Вычисляем DCG@1
dcg_value = dcg_score(dup_ranks, k=1)
print(f"DCG@1 = {dcg_value:.3f}")

# Вычисляем DCG@4
dcg_value = dcg_score(dup_ranks, k=4)
print(f"DCG@10 = {dcg_value:.3f}")

DCG@1 = 0.000
DCG@10 = 0.631


In [16]:
dup_ranks_list = [[i] for i in range(1, 50)]
ans_list = []
for i in dup_ranks_list:
    ans = hits_count(i, 47) - dcg_score(i, 1)
    # print(i, "->", ans)
    ans_list.append(ans)

print(max(ans_list))

1.0


Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [17]:
import pandas as pd

In [18]:
copy_answers = ["How does the catch keyword determine the type of exception that was thrown",]

# наши кандидаты
candidates_ranking = [["How Can I Make These Links Rotate in PHP",
                       "How does the catch keyword determine the type of exception that was thrown",
                       "NSLog array description not memory address",
                       "PECL_HTTP not recognised php ubuntu"],]

# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print('Ваш ответ HIT:', [hits_count(dup_ranks, k) for k in range(1, 5)])
print('Ваш ответ DCG:', [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [19]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame([[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
                               index=['HITS', 'DCG'], columns=range(1,5))
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [22]:
!unzip stackoverflow_similar_questions.zip

Archive:  stackoverflow_similar_questions.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/train.tsv          
  inflating: data/validation.tsv     


Считайте данные.

In [23]:
def read_corpus(filename):
    data = []
    with open(filename, encoding='utf-8') as file:
        for line in file:
            data.append(line.strip().split('\t'))
    return data

Нам понадобиться только файл validation.

In [24]:
validation_data = read_corpus('./data/validation.tsv')

Кол-во строк

In [25]:
len(validation_data)

3760

Размер нескольких первых строк

In [26]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

# Вопрос
print(validation_data[0][0])
# Похожий вопрос
print(validation_data[0][1])
# Отрицательный пример 1
print(validation_data[0][2])
# Отрицательный пример 2
print(validation_data[0][3])
# и т.д.

1 1001
2 1001
3 1001
4 1001
5 1001
How to print a binary heap tree without recursion?
How do you best convert a recursive function to an iterative one?
How can i use ng-model with directive in angular js
flash: drawing and erasing


In [27]:
validation_data[0][0]

'How to print a binary heap tree without recursion?'

### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
from copy import deepcopy

In [29]:
def rank_candidates(question, candidates, embeddings, tokenizer, dim=200):
    """
        question: строка
        candidates: массив строк(кандидатов) [a, b, c]
        result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """
    # Вектор вопроса
    question_vec = np.array([question_to_vec(question, embeddings, tokenizer, dim=200)])
    # Матрица кандидатов
    candidates_vec = np.array([question_to_vec(i, embeddings, tokenizer, dim=200) for i in candidates])
    similarity = cosine_similarity(candidates_vec, question_vec)
    df = pd.DataFrame({"num": list(range(len(candidates))),
                       "candidate": candidates,
                       "similarity": similarity.reshape(-1)})
    df = df.sort_values(by="similarity", ascending=False)
    otput_list = []
    for i, row in df.iterrows():
        otput_list.append((row['num'], row['candidate']))
    return otput_list

    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [30]:
questions = ['converting string to list', 'Sending array via Ajax fails']

candidates = [['Convert Google results object (pure js) to Python object', # первый эксперимент
               'C# create cookie from string and send it',
               'How to use jQuery AJAX for an outside domain?'],

              ['Getting all list items of an unordered list in PHP',      # второй эксперимент
               'WPF- How to update the changes in list item of a list',
               'select2 not displaying search results']]

In [31]:
for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
    for rank in ranks:
        print(rank)
    print()

(1, 'C# create cookie from string and send it')
(0, 'Convert Google results object (pure js) to Python object')
(2, 'How to use jQuery AJAX for an outside domain?')

(1, 'WPF- How to update the changes in list item of a list')
(0, 'Getting all list items of an unordered list in PHP')
(2, 'select2 not displaying search results')



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [32]:
# должно вывести
results = [[(1, 'C# create cookie from string and send it'),
            (0, 'Convert Google results object (pure js) to Python object'),
            (2, 'How to use jQuery AJAX for an outside domain?')],
           [(0, 'Getting all list items of an unordered list in PHP'), #скрыт
            (2, 'select2 not displaying search results'), #скрыт
            (1, 'WPF- How to update the changes in list item of a list')]] #скрыт

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### ***Вопрос 5:***
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?


#### **Ответ 5**

`021`

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [33]:
from tqdm.notebook import tqdm
from gensim.models.keyedvectors import KeyedVectors

In [34]:
q, *ex = validation_data[0]

print(q)
print('-'*20)
for i in range(10):
    print(ex[i])

How to print a binary heap tree without recursion?
--------------------
How do you best convert a recursive function to an iterative one?
How can i use ng-model with directive in angular js
flash: drawing and erasing
toggle react component using hide show classname
Use a usercontrol from another project to current webpage
~ Paths resolved differently after upgrading to ASP.NET 4
Materialize datepicker - Rendering when an icon is clicked
Creating PyPi package - Could not find a version that satisfies the requirement iso8601
How can I analyze a confusion matrix?
How do I declare a C array in Swift?


In [35]:
# Число слов
len(wv_embeddings.index_to_key)

1787145

In [36]:
# wv_embeddings = KeyedVectors.load_word2vec_format("SO_vectors_200.bin?download=1", binary=True)
tokenizer = MyTokenizer() # простой токенизатор

wv_ranking = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data[:max_validation_examples])):
    q, *ex = line
    ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
    wv_ranking.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [37]:
k_values = [1, 5, 10, 100, 500, 1000]

for k in tqdm(k_values):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking, k), k, hits_count(wv_ranking, k)))

df_wv_pretrained_ranking = pd.DataFrame(data={"dcg@k": [dcg_score(wv_ranking, k) for k in k_values],
                                              "hits@k": [hits_count(wv_ranking, k) for k in k_values]},
                                        index=k_values)

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.285 | Hits@   1: 0.285
DCG@   5: 0.342 | Hits@   5: 0.393
DCG@  10: 0.360 | Hits@  10: 0.449
DCG@ 100: 0.406 | Hits@ 100: 0.679
DCG@ 500: 0.431 | Hits@ 500: 0.878
DCG@1000: 0.444 | Hits@1000: 1.000


Из формул выше можно понять, что

- $ \text{Hits@K} $ **монотонно неубывающая функция** $ K $, которая стремится к 1 при $ K \to \infty $.

- $ \text{DCG@K} $ **монотонно неубывающая функция** $ K $, но рост замедляется с увеличением $ K $ из-за убывания веса $ \frac{1}{\log_2(1 + \text{rank}_{q'_i})} $.

### Эмбеддинги, обученные на корпусе похожих вопросов

In [38]:
train_data = read_corpus('./data/train.tsv')

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

***Рассмотрим подробнее*** данное склеивание.

1. Каждая строка из train_data разбивается на вопрос (question) и список кандидатов.

2. Для каждого кандидата вопрос склеивается с ним в одну строку.

3. Склеенная строка (combined_text) токенизируется, и полученный список токенов добавляется в общий корпус (corpus).

***Пример***

    Вопрос: "What is Python?"
    Кандидаты: ["Python is a programming language", "Java is another language"]
    Склеенные строки:
        "What is Python? Python is a programming language"
        "What is Python? Java is another language"
         
    Токенизированные списки:
        ['what', 'is', 'python', 'python', 'is', 'a', 'programming', 'language']
        ['what', 'is', 'python', 'java', 'is', 'another', 'language']
         
     

In [39]:
len(train_data)

1000000

In [40]:
train_data[:5]

[['converting string to list',
  'Convert Google results object (pure js) to Python object'],
 ['Which HTML 5 Canvas Javascript to use for making an interactive drawing tool?',
  'Event handling for geometries in Three.js?'],
 ['Sending array via Ajax fails',
  'Getting all list items of an unordered list in PHP'],
 ['How to insert CookieCollection to CookieContainer?',
  'C# create cookie from string and send it'],
 ['Updating one element of a bound Observable collection',
  'WPF- How to update the changes in list item of a list']]

Склеивание пар текстов для осздания корпуса

In [41]:
train_data[111258]

['Determine if the device is a smartphone or tablet?',
 'Change imageView params in all cards together']

In [42]:
# Создаем общий корпус текстов
corpus = []

tokenizer = MyTokenizer()

for i in train_data:
    words = tokenizer.tokenize(" ".join(i))
    words = [w.lower() for w in words]
    corpus.append(words)

print(len(corpus))
corpus[:2]

1000000


[['converting',
  'string',
  'to',
  'list',
  'convert',
  'google',
  'results',
  'object',
  'pure',
  'js',
  'to',
  'python',
  'object'],
 ['which',
  'html',
  '5',
  'canvas',
  'javascript',
  'to',
  'use',
  'for',
  'making',
  'an',
  'interactive',
  'drawing',
  'tool',
  'event',
  'handling',
  'for',
  'geometries',
  'in',
  'three',
  'js']]

In [43]:
import multiprocessing
import time

cores = multiprocessing.cpu_count()
print(f"n cores = {cores}")

n cores = 2


In [44]:
from gensim.models import Word2Vec

# Варьировать параметры?
vw_parameters = {
    "vector_size": [100, 200, 300],
    "window": [2, 3, 4, 5],
    "min_count": [5, 20, 100]
}

start = time.time()

# Объявляем модель Word2Vec
w2v_model = Word2Vec(
    # sentences=corpus,        # Корпус токенизированных текстов, не пишем, если не хотим сразу обучить
    vector_size=200,         # Размерность векторов
    window=5,                # Размер окна контекста
    min_count=5,             # Минимальная частота слов
    workers=4                # Количество потоков
)

w2v_model.build_vocab(corpus, progress_per=20000)
w2v_model.train(corpus, total_examples=w2v_model.corpus_count, epochs=5, report_delay=1)

embeddings_trained = w2v_model.wv

end = time.time()
print(f"Train time = {end - start}")

Train time = 183.34485936164856


In [45]:
len(embeddings_trained.index_to_key)

37334

In [46]:
# Пример эмбеддинга какого-то слова
embeddings_trained['java']

array([ 3.00604510e+00, -2.02985227e-01,  7.51745582e-01,  8.58520150e-01,
        4.71192271e-01, -1.11467946e+00, -2.56308079e-01,  1.04404521e+00,
        5.12992322e-01, -6.02147639e-01, -5.84536195e-01,  8.51869285e-02,
       -1.15563262e+00, -3.74329954e-01, -8.22721362e-01,  2.20942283e+00,
        7.49025166e-01,  2.13052797e+00,  8.12277079e-01, -6.56099677e-01,
        1.27430761e+00, -6.70877934e-01,  1.10418534e+00, -3.08917403e-01,
        5.93269169e-01,  1.57031035e+00,  2.36446813e-01, -8.69273543e-01,
       -2.96229750e-01,  3.64926308e-01, -2.78650641e-01, -1.95233715e+00,
        1.48585153e+00, -1.01951158e+00,  2.53606009e+00, -1.95217764e+00,
        1.69044757e+00, -4.45440829e-01,  1.16434026e+00, -1.02928594e-01,
        5.13072729e-01,  9.27519426e-02,  1.27408253e-02, -2.54268408e+00,
       -1.12819111e+00,  1.64152384e-01, -1.74454629e-01,  3.85254413e-01,
        2.04755291e-02,  1.64100885e+00, -1.08585918e+00,  1.06940985e+00,
        4.96648327e-02,  

In [47]:
wv_ranking_trained = []
max_validation_examples = 1000
for i, line in enumerate(tqdm(validation_data[:max_validation_examples])):
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer)
    wv_ranking_trained.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [48]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print("DCG@%4d: %.3f | Hits@%4d: %.3f" % (k, dcg_score(wv_ranking_trained, k),
                                              k, hits_count(wv_ranking_trained, k)))

df_wv_trained_ranking = pd.DataFrame(data={"dcg@k": [dcg_score(wv_ranking_trained, k) for k in k_values],
                                           "hits@k": [hits_count(wv_ranking_trained, k) for k in k_values]},
                                     index=k_values)

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.197 | Hits@   1: 0.197
DCG@   5: 0.249 | Hits@   5: 0.296
DCG@  10: 0.265 | Hits@  10: 0.346
DCG@ 100: 0.315 | Hits@ 100: 0.599
DCG@ 500: 0.346 | Hits@ 500: 0.839
DCG@1000: 0.363 | Hits@1000: 1.000


Обученнный на текстах Word2Vec справляется хуже, чем предобученный

In [50]:
df_wv_pretrained_ranking - df_wv_trained_ranking

,dcg@k,hits@k
1,0.088000,0.088
5,0.092883,0.097
10,0.094838,0.103
100,0.090290,0.080
500,0.085256,0.039
1000,0.080961,0.000


# Эксперименты
## Методы токенизации

Попробуем различные методы токенизации

In [51]:
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

import spacy
nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [53]:
import numpy as np
import re
import string
from nltk.tokenize import word_tokenize, WordPunctTokenizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer, PorterStemmer

STOP_WORDS_SET = set(stopwords.words('english'))

class MyTokenizer:
    def __init__(self, stop_words=False, lemmatize="default"):
        self.stop_words = stop_words
        self.lemmatize = lemmatize

    def tokenize(self, text):
        words = re.findall('\w+', text)
        words = [w.lower() for w in words]
        # Удалять стоп слова или нет
        if self.stop_words:
            words = [i for i in words if not i in STOP_WORDS_SET]
        # Какой лемматизатор использовать
        if self.lemmatize == 'default': # Без нормализации
            return words
        if self.lemmatize == 'nltk':
            lemmatizer = WordNetLemmatizer()
            words = [lemmatizer.lemmatize(w) for w in words]
        elif self.lemmatize == 'proter':
            stemmer = PorterStemmer()
            words = [stemmer.stem(word) for word in words]
        elif self.lemmatize == 'spacy':
            spacy_results = nlp(text)
            words_out = []
            for token in spacy_results:
                if not token.lemma_ in string.punctuation:
                    if self.stop_words and token.lemma_ in STOP_WORDS_SET:
                        continue
                    words_out.append(token.lemma_)
            return words_out
        return words


lemmatizers = ['default', 'nltk', 'proter', 'spacy']
tokenizer = MyTokenizer(stop_words=True, lemmatize='default')
test_text = "This is a test text. My freinds love playing tabletop games (I don't)"
tokenizer.tokenize(test_text)

['test', 'text', 'freinds', 'love', 'playing', 'tabletop', 'games']

Посмотрим, есть ли разница в лемматизации

In [54]:
for i in lemmatizers:
    print(i)
    print('---')
    for sw in [False, True]:
        tokenizer = MyTokenizer(stop_words=sw, lemmatize=i)
        print(" ".join(tokenizer.tokenize(test_text)))
    print('='*30)

default
---
this is a test text my freinds love playing tabletop games i don t
test text freinds love playing tabletop games
nltk
---
this is a test text my freinds love playing tabletop game i don t
test text freinds love playing tabletop game
proter
---
thi is a test text my freind love play tabletop game i don t
test text freind love play tabletop game
spacy
---
this be a test text my freind love play tabletop game I do not
test text freind love play tabletop game I


In [71]:
df_results = pd.DataFrame()
max_validation_examples = 1000

for sw in [True, False]:
    for l in lemmatizers:
        if l == "spacy":
            continue

        print(f"Lemmatizer: {l}, stop words = {sw}")
        tokenizer = MyTokenizer(stop_words=sw, lemmatize=l)

        wv_ranking = []
        for i, line in enumerate(tqdm(validation_data[:max_validation_examples])):
            q, *ex = line
            ranks = rank_candidates(q, ex, wv_embeddings, tokenizer)
            wv_ranking.append([r[0] for r in ranks].index(0) + 1)


        df_current = pd.DataFrame(
            data={"k": k_values,
                "dcg@k": [dcg_score(wv_ranking, k) for k in k_values],
                "hits@k": [hits_count(wv_ranking, k) for k in k_values]})
        df_current['lemmatizer'] = l
        df_current['stop_words'] = sw

        df_results = pd.concat([df_results, df_current])

Lemmatizer: default, stop words = True


  0%|          | 0/1000 [00:00<?, ?it/s]

Lemmatizer: nltk, stop words = True


  0%|          | 0/1000 [00:00<?, ?it/s]

Lemmatizer: proter, stop words = True


...

  0%|          | 0/1000 [00:00<?, ?it/s]

Lemmatizer: nltk, stop words = False


  0%|          | 0/1000 [00:00<?, ?it/s]

Lemmatizer: proter, stop words = False


  0%|          | 0/1000 [00:00<?, ?it/s]

In [74]:
df_results = df_results.reset_index(drop=True)

cut = df_results['k'] == 10

df_results[cut]

,k,dcg@k,hits@k,lemmatizer,stop_words
2,10,0.524898,0.650,default,True
8,10,0.514400,0.633,nltk,True
14,10,0.423063,0.536,proter,True
20,10,0.524405,0.650,default,False
26,10,0.513157,0.632,nltk,False
32,10,0.425820,0.536,proter,False


In [83]:
for k in k_values:
    cut = df_results['k'] == k
    i_max = df_results[cut]['dcg@k'].argmax()
    idx_max = df_results[cut]['dcg@k'].index[i_max]

    print(f"Best dcg@{k} -> {df_results.loc[idx_max, 'lemmatizer'], df_results.loc[idx_max, 'stop_words']}")

Best dcg@1 -> ('default', True)
Best dcg@5 -> ('default', True)
Best dcg@10 -> ('default', True)
Best dcg@100 -> ('default', True)
Best dcg@500 -> ('default', True)
Best dcg@1000 -> ('default', True)


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:

* Какой принцип токенизации даёт качество лучше и почему?

Наилучшего качества позволяет достичь очситка текста от пунктуации, замтем приведение слов к нижнему регистру

* Помогает ли нормализация слов?

Нормализация слов не улучшает качество ранжирования, лемматизация немного ухудшает, стемминг ухудшает значительно

* Какие эмбеддинги лучше справляются с задачей и почему?

Лучше справляются предобученные эмбеддинги, так как они обучались на бОльшем объеме текстов

* Почему получилось плохое качество решения задачи?

Судя по всему, такой алгоритм ставит на первые места предложения с наибольших количеством похожих слов, не всегда учитывая смысл предложения целиком.

* Предложите свой подход к решению задачи.

1. Можно попробовать обучать эмбеддинги предложений или n-грам
2. Использовать другую модель для векторизации слов или текстов